# Tarea audio 2
Autor: Roberto Vázquez Maestre

In [ ]:
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt

## Definiciones

- **Frecuencia de muestreo:** Es la cantidad de veces por segundo que se mide una señal analógica para convertirla en digital. Determina qué tan finamente se representa el tiempo de la señal y establece el límite máximo de frecuencia que puede capturar.  

- **Aliasing:** Es un fenómeno de distorsión que ocurre cuando la señal se muestrea a una frecuencia insuficiente. Dando lugar a que esta ni pueda ser reconstruidad de forma correcta.

- **Profundidad de bits:** Es la cantidad de bits utilizados para representar cada muestra de la señal. Determina la resolución de amplitud de la señal digital y afecta directamente el nivel de ruido de cuantización y la fidelidad del audio.  

- **Ancho de banda:** Es el rango de frecuencias que una señal puede contener o que un sistema puede transmitir. En audio digital, está limitado por la mitad de la frecuencia de muestreo (teorema de Nyquist).  

- **Tasa de bits:** Es la cantidad de datos que se procesan o transmiten por segundo en un archivo digital. Depende de la frecuencia de muestreo, la profundidad de bits y el número de canales, y define la calidad y tamaño del archivo digital.

In [ ]:
cwd = os.getcwd()
audio_input_path_est = os.path.join(cwd, os.path.join('Audio_input'))  
audio_input_path_mono = os.path.join(cwd, os.path.join('Audio_output'))
audio_output_path = os.path.join(cwd, os.path.join('Audio_output')) 
print(f'Directorios con los audios de entrada: {audio_input_path_est} y {audio_input_path_mono}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')

In [ ]:
filename = os.path.join(audio_input_path_est, 'game_of_thrones.wav')
sample_rate, audio_data_est = wavfile.read(filename)

filename = os.path.join(audio_input_path_mono, 'game_of_thrones_mono.wav')
sample_rate, audio_data_mono = wavfile.read(filename)

print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

Aquí se muestra la gráfica en el dominio del tiempo para el audio mono y estéreo

In [ ]:

filename = os.path.join(audio_input_path_est, 'game_of_thrones.wav')
sample_rate, audio_data_est = wavfile.read(filename)

filename = os.path.join(audio_input_path_mono, 'game_of_thrones_mono.wav')
sample_rate, audio_data_mono = wavfile.read(filename)

print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

if len(audio_data_est.shape) > 1:
    audio_data_est = audio_data_est[:, 0]

t_est = np.arange(len(audio_data_est)) / sample_rate
t_mono = np.arange(len(audio_data_mono)) / sample_rate

fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

end = 1000

# Señal estéreo (visualizamos solo el canal 1)
ax[0].plot(t_est[:end], audio_data_est[:end], marker='o', markevery=20)
ax[0].set_title('Audio estéreo (canal 1) en el dominio del tiempo')
ax[0].set_ylabel('Amplitud')
ax[0].grid(True)

# Señal mono
ax[1].plot(t_mono[:end], audio_data_mono[:end], c='tab:red', marker='o', markevery=20)
ax[1].set_title('Audio mono en el dominio del tiempo')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Amplitud')
ax[1].grid(True)

plt.tight_layout()
plt.show()

Se realiza la Transformada rápida de Fourier (FFT) sobre el audio mono. 

Se convierte el audio a mono para simplificar el análisis y se aplica la FFT para ver la composición en frecuencia del sonido, es decir, qué tonos y sonidos forman la señal y cuánto pesan, algo que no se ve en la forma de onda en el tiempo. Esto ayuda a detectar ruidos, identificar sonidos predominantes y mejorar o modificar el audio.

In [ ]:


n = len(audio_data_mono)
Fs = sample_rate

ch_Fourier = np.fft.fft(audio_data_mono)

abs_ch_Fourier = np.abs(ch_Fourier[:n//2])

frequencies = np.linspace(0, Fs/2, n//2)

# Graficamos
plt.figure(figsize=(12,5))
plt.plot(frequencies, abs_ch_Fourier)
plt.title('Espectro de Frecuencia - Audio Mono')
plt.ylabel('Amplitud')
plt.xlabel('Frecuencia (Hz)')
plt.grid(True)
plt.show()

Se calcula la energia del espectro y se elige un epsilon por el cual se hace el corte. En este caso se ha elegido el epsilon de .041, debido a que esto elimina algunas frecuencias altas de poca energía, donde no hay mucho contenido importante, dejando solo la parte principal del espectro. 

In [ ]:
# Definimos diferentes epsilons (energía que NO conservamos)
eps_values = [1e-5, .02, .041, .063, .086, .101, .123]

# Elegimos uno (puedes ir cambiando el índice)
eps = eps_values[2]
print(f'Epsilon: {eps}')

# Valor de corte de energía acumulada
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energía del espectro: {thr_spec_energy}')

# Energía acumulada
spec_energy = np.cumsum(abs_ch_Fourier)

# Máscara booleana
frequencies_to_remove = spec_energy > thr_spec_energy
print(f'Máscara: {frequencies_to_remove}')

# Índice donde se alcanza el umbral
cut_index = np.argmax(frequencies_to_remove)

# Frecuencia de corte
f0 = frequencies[cut_index]
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos
plt.figure(figsize=(12,5))
plt.plot(frequencies, abs_ch_Fourier)
plt.axvline(f0, color='r', label=f'f0 = {int(f0)} Hz')
plt.ylabel('Amplitud')
plt.xlabel('Frecuencia (Hz)')
plt.title('Espectro y frecuencia de corte')
plt.legend()
plt.grid(True)
plt.show()

Se comprime la onda y se guarda en un nuevo archivo con el audio comprimido. El factor de downsampling se calcula a partir del sample_rate.

In [ ]:
# Calculamos el factor D de downsampling.
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos (slicing with stride).
new_data = audio_data_mono[::D]

# Definimos el nombre del audio comprimido generado.
wav_compressed_file = "game_of_thrones_compressed.wav"

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

# Cargamos el nuevo archivo.
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(audio_output_path, wav_compressed_file))

Se muestan los epectogramas de los dos archivos el normal y el comprimido. COmo se puede ver en el comprimido hay muchas más rayas lo que indica que el audio esta mas comprimido y no es del todo continuo.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(audio_data_mono, NFFT=1024, Fs=sample_rate, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

Y se muestan el tamaño de los archivos de audio como se puede ver el comprimido ocupa menos.

In [ ]:
print('El tamaño del archivo de audio en mono sin comprimir es:')
!ls -sh Audio_output/game_of_thrones_mono.wav
print('El tamaño del archivo de audio en mono comprimido es:')
!ls -sh Audio_output/game_of_thrones_compressed.wav

In [ ]:
IPython.display.Audio(audio_data_mono, rate=sample_rate)

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)